In [9]:
## Importing Libraries ## 
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from IPython.display import Markdown as md
import warnings
warnings.filterwarnings('ignore')
from scipy.optimize import minimize


In [10]:
## Defining Classes and Stuff ## 

#############################
class PoissonProcess():
    def __init__(self, lam, T):
        self.lam = lam
        self.T = T
        self.simulate()

    def simulate(self, method='inter_arrival_time'):
        if method == 'inter_arrival_time':
            N = int(self.lam * self.T * 1.3)
            inter_ls = np.random.exponential(1/self.lam, size=N)
            arrival_time_ls = np.cumsum(inter_ls)
            self.arrival_time_ls = arrival_time_ls[arrival_time_ls <= self.T]
        if method == 'uniformity_property':
            N = np.random.poisson(self.T * self.lam)
            arrival_time_ls = np.random.uniform(0, self.T, size=N)
            self.arrival_time_ls = np.sort(arrival_time_ls)

    def get_arrival_time(self):
        return self.arrival_time_ls

    def print_parameter(self):
        print('lambda = {}, T = {}'.format(self.lam, self.T))

    def N_t(self, t):
        assert t >= 0
        assert t <= self.T
        if t == 0:
            return 0
        else:
            return np.argmax(self.arrival_time_ls > t)

    def plot_N_t(self, color='r',alpha=1):
        positive_inf = max(self.arrival_time_ls) * 1.2
        negative_inf = - max(self.arrival_time_ls) * 0.1
        n_arrival = len(self.arrival_time_ls)
        x_ls = np.concatenate([[negative_inf, 0], np.repeat(self.arrival_time_ls,2), [positive_inf]])
        y_ls = np.concatenate([[0], np.repeat(np.arange(n_arrival + 1),2)])
        plt.plot(x_ls, y_ls, c=color, alpha=alpha)

#############################

___

## Implementing Rejection Sampling

In [11]:
## Loading in November Arrival Data ## 
rates = pd.read_csv('november_arrival_data.csv')
rates['day'] = pd.to_datetime(rates['date'])
rates = rates.drop('date', axis = 1)
rates = rates[['day', 'open', 'peak', 'afternoon']]
rates = rates.rename( columns = {'day' : 'Date', 
                                 'open' : '8', 
                                 'peak' : '11',
                                 'afternoon' : '4', })
rates = rates.dropna()
rates.head(5)

,Date,8,11,4
0,2019-11-01,1517,3330,3090.0
1,2019-11-02,1540,2820,3870.0
2,2019-11-03,1517,3210,3090.0
3,2019-11-04,1657,3000,2820.0
4,2019-11-05,1517,3180,2640.0


In [12]:
# Removing Outliers 
rates2 = rates.drop('Date', axis = 1)
q1 = rates2.quantile(0.25)
q3 = rates2.quantile(0.75)
IQR = q3 - q1
LB = q1 - 1.25*IQR
UB = q3 + 1.25*IQR

rates_in_8 = rates2[ (rates2['8'].between(LB[0], UB[0])) ][['8']]
rates_out_8 = rates2[ ~(rates2['8'].between(LB[0], UB[0])) ][['8']]

rates_in_11 = rates2[ (rates2['11'].between(LB[1], UB[1])) ][['11']]
rates_out_11 = rates2[ ~(rates2['11'].between(LB[1], UB[1])) ][['11']]

rates_in_4 = rates2[ (rates2['4'].between(LB[2], UB[2])) ][['4']]
rates_out_4 = rates2[ ~(rates2['4'].between(LB[2], UB[2])) ][['4']]

rates_in = pd.concat([rates_in_8,rates_in_11, rates_in_4], ignore_index=True, axis=1)
rates_out = pd.concat([rates_out_8,rates_out_11, rates_out_4], ignore_index=True, axis=1)

In [13]:
# Calculating Arrival Rate for Model # 1
rates_in_flat = rates_in.values.flatten()
rates_in_flat = rates_in_flat[~np.isnan(rates_in_flat)]
print('Model #1 Arrival Rate: ', np.median(rates_in_flat), ' customers/hr')

Model #1 Arrival Rate:  2880.0  customers/hr


In [14]:
# Means & Medians after Removing Outliers

old_means = [np.mean(rates2.iloc[:,i]) for i in np.arange(0,3)]
old_medians = [np.median(rates2.iloc[:,i]) for i in np.arange(0,3)]

new_means = [np.mean(rates_in[i].dropna()) for i in np.arange(0,3)]
new_medians = [np.median(rates_in[i].dropna()) for i in np.arange(0,3)]

print('Old Means: ', np.round(old_means, 2))
print('New Means: ', np.round(new_means,2))

print('Old Medians: ', np.round(old_medians, 2))
print('New Medians: ', np.round(new_medians,2))

Old Means:  [1747.68 3540.   3372.86]
New Means:  [1651.28 3458.89 3168.  ]
Old Medians:  [1610. 3180. 3030.]
New Medians:  [1587. 3180. 2970.]


In [15]:
## Find the Coefficients for our Fourth-Order Arrival Rate Function ## 
p1 = [0.5, np.round(new_medians[0], 3)] #8:00 am
p2 = [3.5, np.round(new_medians[1], 3)] #11:00 am
p3 = [5.5, np.round(new_medians[1], 3)] #1:00 pm
p4 = [8.5, np.round(new_medians[2], 3)] #4:00 pm
p5 = [11.5,np.round(new_medians[0], 3)] #7:00 pm

A = [[p1[0]**i for i in np.arange(5)], [p2[0]**i for i in np.arange(5)], [p3[0]**i for i in np.arange(5)],
     [p4[0]**i for i in np.arange(5)], [p5[0]**i for i in np.arange(5)]]
B = [p1[1], p2[1], p3[1], p4[1], p1[1]]
x = np.linalg.inv(A).dot(B)

In [16]:
[p1[0]**i for i in np.arange(5)]

[1.0, 0.5, 0.25, 0.125, 0.0625]

In [17]:
## Time-Varying Arrival Rate Function ## 
def ArrivalRate_Function(t):
    return x[0] + x[1]*t + x[2]*t**2 + x[3]*t**3 + x[4]*t**4

In [18]:
## Rejection Sampling to Simulate a Time-Inhomogeneous Poisson Process (arrival rate varies continuously) ## 
def RejectionSampling(rate_function):
    
    ## Differentiation of Rate Function  ## 
    optimized = minimize(lambda t: -rate_function(t), 0, tol=1e-9)
    Max_ArrRate = -optimized.fun
    timeof_Max_ArrRate = optimized.x
    
    ## Original Arrival List (simulated with the maximum rate)  ## 
    original_process = PoissonProcess(lam = Max_ArrRate, T = 11.999)
    original_arrival_ls = original_process.get_arrival_time()
    num_original_arrivals = len(original_arrival_ls)
    #print("num_original_arrivals = ",num_original_arrivals)
    
    ## Thinning Process  ## 
    accepted_arrival_ls = [] 
    
    for arr_time in original_arrival_ls:
        u = np.random.uniform(0,1)
        keep_prob = np.round(rate_function(arr_time),3) / round(Max_ArrRate,3)        
        if abs(keep_prob<0.00001):
            keep_prob=0
            
        binom = np.random.binomial(n = 1, p = keep_prob)
        if binom:
            accepted_arrival_ls = np.append(accepted_arrival_ls, np.round(arr_time,3))
        
    return np.round(accepted_arrival_ls,3)

In [19]:
## Iteratively generate 30 days of arrivals to use for models ## 
for day in np.arange(1,31):
    arrivals = RejectionSampling(ArrivalRate_Function)
    dict = {"ArrivalTime":arrivals}
    df = pd.DataFrame(dict)    
    df.to_csv('data/day'+str(day)+'_arrivals.csv', index=False)